What to do ?
done with forward pass 
- implement backward pass `fit method`
- multi dimensional input

In [297]:
import numpy as np
from tabulate import tabulate
import time
import mercury as mr

In [298]:
input_ = np.random.randint(2, 5, (30))

In [299]:
input_

array([3, 2, 2, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 4, 3, 2, 2, 4, 4, 3, 3, 4,
       2, 2, 3, 4, 3, 2, 4, 3])

In [300]:
class Input:
    
    '''
    making it for only one dimensional input
    '''
    def __init__(self, input_shape:int, layer_name:str):
        
        self.layer_name = layer_name
        self.units = input_shape
        self.input_data:np.array = np.zeros(self.units) # None only at time of initialization
        
        # these parameters doesnt exist for input layer
        self.previous_layer = None
        self.previous_layer_units = None
        self.__weights = None
        self.__biases = None
        
        
    def forward_pass(self):
        return self.input_data

In [301]:
class DensePhaseThree:
    
    def __init__(self, units:int, activation:str, layer_name:str):
        self.units = units
        self.activation = activation
        self.layer_name = layer_name
        
        self.previous_layer:DensePhaseThree = None
        self.previous_layer_values = None
        self.previous_layer_units = None
        self.__weights = None
        self.__biases = None
        
        
    
    def forward_pass(self) -> np.array:

        #  determining previous layer units
        
        self.previous_layer_units = self.previous_layer.units
        
        # if the previous layer is input layer then previous layer values will be input data
        if isinstance(self.previous_layer, Input):
            self.previous_layer_values = self.previous_layer.input_data

        # if the previous layer is dense layer then previous layer values will be activation output of previous layer
        if isinstance(self.previous_layer, DensePhaseThree):
            self.previous_layer_values = self.previous_layer.activation_output

                
        # initializing weights and biases of this layer
        self.__weights, self.__biases = self.__initialize_weights_and_biases(self.previous_layer_units)
        
        
        # calculating output ---> WiXi + Bo 
        self.output = np.add(np.dot(self.__weights, self.previous_layer_values), self.__biases)


        # applying activation function on output
        if self.activation.lower() == 'relu':
            activation_func = np.vectorize(self.__relu_activation)
            self.activation_output = activation_func(self.output).astype(np.float32)
            
        elif self.activation.lower() == 'sigmoid':
            activation_func = np.vectorize(self.__sigmoid_activation)
            self.activation_output = activation_func(self.output).astype(np.float32)
            
        else: # linear
            self.activation_output = self.output
            
        print("============ Layer Name: ", self.layer_name.upper(), " ============")
        print("\n\ncurrent_layer_units = ", self.units, "\n")
        print("biases_shape = ", self.__biases.shape, "\n")
        print("weights_shape = ", self.__weights.shape, "\n")
        print("previous layer units = ", self.previous_layer_units, "\n")
        print("previous layer values = ", self.previous_layer_values, "\n")
        print("Output for next layer is =", self.activation_output, "\n")
        print("===============================================")
        

        
        return self.activation_output

        
    def __initialize_weights_and_biases(self,previous_layer_units:int):
        
        '''
        Note: using `he normal` initialization
        '''
        
        biases = np.random.randn(self.units)*np.sqrt(2/previous_layer_units)
        weights = np.random.randn(self.units, previous_layer_units)*np.sqrt(2/previous_layer_units)
        
        return weights, biases
        

    def __relu_activation(self, layer_output:np.array):
        return max(0, layer_output)
    
    
    def __sigmoid_activation(self, layer_output:np.array):
        if layer_output>0:
            return 1/(1+np.exp(-layer_output)) # 1/(1+e^-z)
        else:
            return np.exp(layer_output)/(1+np.exp(layer_output)) # e^z / (1+e^z)
    

In [302]:
class Model:
    
    def __init__(self):
        self.layers = []
        self.count = 1
        
        
               
    def add(self, layer:DensePhaseThree):
        layer.previous_layer = None
        '''
        set last item in list as previous layer
        '''
        self.layers.append(layer)

    def show_layers(self):
        print("=================== LAYER INFO ===================")
        for index, layer in enumerate(self.layers):
            print(f"""
                  Index::{index}
                        Layer Name:: {layer.layer_name} {type(layer.layer_name)}
                        Layer Address:: {layer}  {type(layer)}      
                        Layer units::{layer.units} {type(layer.units)}
                        Previous Layer::{layer.previous_layer} {type(layer.previous_layer)}
                        
                        """)
                
            
    def compile_(self, input_data:np.array): 
        '''
        connecting all the layers
        
        running forward pass for all layers
        
        and connect input layer with input data
        '''       
        
        #  connecting all the layers
        
        for i in range(len(self.layers)-1):
            # print(f"set {self.layers[i].layer_name}::{self.layers[i]} as previous layer of {self.layers[i+1].layer_name}::{self.layers[i+1]}")
            self.layers[i+1].previous_layer = self.layers[i]

        # print("\n\nDONE WITH CONNECTING ALL THE LAYERS \n\n")
        
        if isinstance(self.layers[0], Input) and self.layers[0].units == input_data.shape[0]:
            self.layers[0].input_data = input_data
                
        for layer in self.layers:
            layer.forward_pass() 
            
            
    def loss_calc(self, y_true:np.array, y_pred:np.array):
        '''
        calculating mean squared error
        '''
        return np.mean(np.square(y_true - y_pred))   
            
    def fit(self):
        '''
        initialize weights and biases with 0 & 1,
        take weights and add 100 to it, 
        do it 5 times(epochs) and
        dont update  weights
        # '''
        for layer in reversed(self.layers):
            if isinstance(layer, Input):
                print("no weights and biases update for input layer\n")
                continue
            
            print("weights before updating = ", layer._DensePhaseThree__weights, "\n")
            layer._DensePhaseThree__weights = layer._DensePhaseThree__weights * 100
            print("weights after updating = ", layer._DensePhaseThree__weights, "\n")
            time.sleep(1)
            
            print("biases before updating = ", layer._DensePhaseThree__biases, "\n")
            layer._DensePhaseThree__biases = layer._DensePhaseThree__biases + 200
            print("biases after updating = ", layer._DensePhaseThree__biases, "\n")
            print("\n==============================\n")
            
            
            

In [303]:
model= Model()

In [304]:
model.add(Input(input_shape=input_.shape[0], layer_name="input_"))
model.add(DensePhaseThree(units = 1024,activation = "relu", layer_name="first"))
model.add(DensePhaseThree(units = 512,activation = "relu", layer_name="second"))
model.add(DensePhaseThree(units = 2,activation = "SIGMOID", layer_name="third"))

In [305]:
model.compile_(input_)

============ Layer Name:  FIRST  ============


current_layer_units =  1024 

biases_shape =  (1024,) 

weights_shape =  (1024, 30) 

previous layer units =  30 

previous layer values =  [3 2 2 3 3 3 2 2 3 3 3 3 3 4 3 2 2 4 4 3 3 4 2 2 3 4 3 2 4 3] 

Output for next layer is = [2.940445  0.        5.4812737 ... 0.        0.        0.       ] 

============ Layer Name:  SECOND  ============


current_layer_units =  512 

biases_shape =  (512,) 

weights_shape =  (512, 1024) 

previous layer units =  1024 

previous layer values =  [2.940445  0.        5.4812737 ... 0.        0.        0.       ] 

Output for next layer is = [ 0.71260387  0.          5.0316224   7.2424097   0.          1.3005242
  0.          0.          0.          0.24294141  0.          2.3072357
  0.46790725  6.894556    0.          0.81726557  3.2096462   2.1931748
  0.          6.215185    4.165792    0.          0.          0.
  0.          3.1342254   0.5504566   0.          0.9520564   0.
  0.          0.      

In [306]:
model.fit()

weights before updating =  [[-0.03038823 -0.02773302 -0.00575415 ...  0.00771625 -0.01516745
  -0.09889268]
 [-0.06693592  0.01793943 -0.08872091 ... -0.0769863   0.06780039
   0.02913393]] 

weights after updating =  [[-3.0388232  -2.77330225 -0.57541508 ...  0.77162467 -1.51674508
  -9.88926784]
 [-6.69359165  1.7939432  -8.87209067 ... -7.69862959  6.7800391
   2.91339276]] 

biases before updating =  [ 0.05613689 -0.0851112 ] 

biases after updating =  [200.05613689 199.9148888 ] 



weights before updating =  [[-0.03839465 -0.02182454  0.0229833  ...  0.05208794  0.00650265
  -0.02857889]
 [ 0.00536811  0.02777356  0.04865701 ...  0.01689733 -0.07412545
  -0.00363756]
 [ 0.02625414  0.11490749  0.04910597 ... -0.05201946  0.00996389
  -0.00360895]
 ...
 [ 0.08129636 -0.0019764   0.06947567 ... -0.0229606   0.00867594
   0.00577343]
 [ 0.00089338 -0.00518507  0.05173811 ...  0.03374507 -0.04190707
   0.05553855]
 [ 0.00186204  0.03127229  0.01459276 ...  0.06932744 -0.01573538
  -0

In [307]:
1024*521

533504